<a href="https://colab.research.google.com/github/miniyollo/mycaptain/blob/main/OCRLaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install selenium
!pip install openpyxl
!pip install pytesseract
!pip install webdriver_manager


In [25]:
# Install Chrome Browser (required for selenium to work)
!apt-get update
!apt-get install google-chrome-stable -y

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not s

In [26]:
import time
import csv
import openpyxl
import pytesseract
from PIL import Image
import io
import base64
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service # Import Service
from selenium.common.exceptions import TimeoutException

# Set up Tesseract path (IMPORTANT: Change if needed)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Windows path

# Advocate and Court Details
STATE = "Gujarat"
DISTRICT = "Ahmedabad"
ADVOCATE_CODE = "G/1818/1999"

# Settings
MAX_CAPTCHA_TRIES = 10  # Higher tries to handle difficult CAPTCHA

# Get the path of the installed chromedriver executable
driver_path = ChromeDriverManager().install()
# Create a Service object with the driver path
service = Service(driver_path)
# Pass the service object to the webdriver.Chrome constructor
driver = webdriver.Chrome(service=service)
driver.maximize_window()

wait = WebDriverWait(driver, 30)

try:
    # Open eCourts Services Website
    driver.get('https://services.ecourts.gov.in/ecourtindia_v6/')

    # Click on "Case Status"
    wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'Case Status'))).click()

    # Click on "Advocate"
    wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'Advocate'))).click()

    # Select State
    state_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'state_code')))
    Select(state_dropdown).select_by_visible_text(STATE)

    # Select District
    time.sleep(2)  # Wait for district dropdown to populate
    district_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'dist_code')))
    Select(district_dropdown).select_by_visible_text(DISTRICT)

    # Enter Advocate Bar Registration Number
    advocate_input = wait.until(EC.presence_of_element_located((By.ID, 'advocateBarRegNo')))
    advocate_input.clear()
    advocate_input.send_keys(ADVOCATE_CODE)

    # Solve CAPTCHA Automatically
    success = False
    for attempt in range(1, MAX_CAPTCHA_TRIES + 1):
        print(f"\nAttempt {attempt} to solve CAPTCHA...")

        # Capture CAPTCHA Image
        captcha_img = driver.find_element(By.ID, 'captcha_image')
        captcha_base64 = captcha_img.screenshot_as_base64

        # Decode and OCR the CAPTCHA
        captcha_data = base64.b64decode(captcha_base64)
        captcha_image = Image.open(io.BytesIO(captcha_data))

        captcha_text = pytesseract.image_to_string(captcha_image, config='--psm 7')
        captcha_text = ''.join(filter(str.isalnum, captcha_text))  # Clean text
        print(f"Detected CAPTCHA: {captcha_text}")

        # Fill the CAPTCHA
        captcha_input = driver.find_element(By.ID, 'captcha')
        captcha_input.clear()
        captcha_input.send_keys(captcha_text)

        # Submit the form
        submit_btn = driver.find_element(By.ID, 'submitFormButton')
        submit_btn.click()

        # Wait and check if submission successful
        time.sleep(5)
        if "Invalid Captcha" in driver.page_source or "Captcha Invalid" in driver.page_source:
            print("Wrong CAPTCHA detected. Refreshing CAPTCHA...")
            refresh_captcha = driver.find_element(By.ID, 'refreshCaptcha')
            refresh_captcha.click()
            time.sleep(2)
            continue  # Retry
        elif driver.find_elements(By.XPATH, "//table[@class='table table-bordered']//tbody/tr") or "No record found" in driver.page_source:
            print("CAPTCHA solved successfully!")
            success = True
            break
        else:
            print("Unexpected page, trying again...")
            refresh_captcha = driver.find_element(By.ID, 'refreshCaptcha')
            refresh_captcha.click()
            time.sleep(2)

    if not success:
        raise Exception("Failed to solve CAPTCHA after multiple attempts.")

    # Fetch Cases
    cases = driver.find_elements(By.XPATH, "//table[@class='table table-bordered']//tbody/tr")

    # Prepare data for Excel
    data = []

    print("\nFound Cases:")
    if not cases:
        print("No cases found or incorrect Advocate Code.")
    else:
        for idx, case in enumerate(cases, 1):
            try:
                case_type = case.find_element(By.XPATH, "./td[1]").text
                case_number = case.find_element(By.XPATH, "./td[2]").text
                filing_date = case.find_element(By.XPATH, "./td[3]").text
                status = case.find_element(By.XPATH, "./td[5]").text
                next_hearing_date = case.find_element(By.XPATH, "./td[7]").text
                court_name = case.find_element(By.XPATH, "./td[8]").text

                print(f"{idx}. {case_number} | Next Hearing: {next_hearing_date}")

                data.append([case_type, case_number, filing_date, status, next_hearing_date, court_name])
            except Exception as e:
                print(f"Error reading a case row: {e}")

    # Save to Excel
    if data:
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet.title = "Advocate Cases"

        # Header
        headers = ["Case Type", "Case Number", "Filing Date", "Status", "Next Hearing Date", "Court Name"]
        sheet.append(headers)

        # Data
        for row in data:
            sheet.append(row)

        # Make header bold
        for cell in sheet[1]:
            cell.font = openpyxl.styles.Font(bold=True)

        # Auto column width
        for column_cells in sheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            sheet.column_dimensions[column_cells[0].column_letter].width = length + 2

        # Save Excel file
        workbook.save("advocate_cases.xlsx")
        print("\nData saved successfully to 'advocate_cases.xlsx'.")

except Exception as e:
    print(f"Error: {e}")

finally:
    driver.quit()

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x5800ddeab33a <unknown>
#1 0x5800dd9296e6 <unknown>
#2 0x5800dd964f7d <unknown>
#3 0x5800dd95f6ce <unknown>
#4 0x5800dd9afd73 <unknown>
#5 0x5800dd9af426 <unknown>
#6 0x5800dd9a15b3 <unknown>
#7 0x5800dd96dcc2 <unknown>
#8 0x5800dd96e981 <unknown>
#9 0x5800dde6dba8 <unknown>
#10 0x5800dde71a17 <unknown>
#11 0x5800dde550c9 <unknown>
#12 0x5800dde725c5 <unknown>
#13 0x5800dde3a90f <unknown>
#14 0x5800dde97898 <unknown>
#15 0x5800dde97a73 <unknown>
#16 0x5800ddeaa2d3 <unknown>
#17 0x7a0ccaf9dac3 <unknown>
